<a href="https://colab.research.google.com/github/marianamartins228/ProjetoBootcampRestart2025/blob/main/Projeto_Bootcamp_Restart_Data_Girls_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd